# Hypothesis Testing for Two Independent Means

---

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.power import TTestIndPower

plt.style.use("fivethirtyeight")
%matplotlib inline

In [2]:
def sample_creator(n, mean, std):
    sample = stats.norm.rvs(loc=0, scale=std, size=n)
    sample = sample - np.mean(sample)
    sample = sample * (std/sample.std(ddof=1))
    return sample + mean

---

Per fare inferenza sulla differenza fra due medie indipendenti, ci sono delle condizioni da rispettare:

- **Independence within groups**: Le osservazioni raccolte nei *sample* devono essere indipendenti. E' molto probabile che si verifichi nel caso in cui sia stato applicato il *random sampling* (in un *observational study*) o il *random assignment* (in un *experiment*). Nel caso di *random sampling* senza *replacement*, i due *sample size* $n_1$ e $n_2$ devono essere entrambi inferiori rispetto al 10% della popolazione.


- **Independence between groups**: I gruppi devono essere indipendenti fra loro, cioè le osservazioni al loro interno non devono essere *paired*  fra un gruppo e l'altro.


- **Sample size/skew**: Maggiore skewness necessita di *sample size* altrettanto alti.

Per calcolare la *test statistic* relativa alla differenza fra **two independent means**, la *point estimate* è la differenza fra le due medie, mentre lo *standard error* va calcolato come la radice quadrata della somma delle varianze dei due gruppi divise per il relativo *sample size*. Infine, i *degrees of freedom* della *t-distribution* in questo caso sono pari al minimo fra i due *sample size* meno uno. Questa è la cosiddetta **Welch Procedure**.

$$\large SE_{(\bar{x_1}-\bar{x_2})}=
\sqrt{\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}}$$

$$\large df=min(n_1-1,\; n_2-1)$$

La *test statistic* è un *t-score* calcolato come segue:

$$\large T=\frac{(\bar{x_1}-\bar{x_2})-null}{SE_{(\bar{x_1}-\bar{x_2})}}$$

In realtà, la formula originale per il calcolo dei *df* è:

$$\large df=\frac{\left(\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}\right)^2}
{\frac{1}{n_1-1}\left(\frac{s_1^2}{n_1}\right)^2+
\frac{1}{n_2-1}\left(\frac{s_2^2}{n_2}\right)^2}$$

Ricordiamo, inoltre, che la differenza $\mu_1-\mu_2$ spesso prende il nome di **Treatment Effect**, perché il più delle volte si basa sulla differenza di valori medi di statistiche di un treatment group e di un control group.

---

Usando la **pooled variance procedure** si assume che entrambe le popolazioni abbiano la stessa varianza.

Il calcolo dello *standard error* cambia così:

$$\large SE(\bar{X_1}-\bar{X_2})=s_p\sqrt{\frac{1}{n_1}+\frac{1}{n_2}}$$

Dove $s_p$ è la *sample pooled standard deviation*, cioè l'*estimator* di $\sigma$, la deviazione standard comune ad entrambe le popolazioni:

$$\large s_p^2=\frac{(n_1-1)s_1^2+(n_2-1)s_2^2}{n_1+n_2-2}$$

I gradi di libertà, in questo caso, devono coincidere con quelli usati nel calcolo della *pooled sample variance*:

$$\large df=n_1-n_2-2$$

La formula per il calcolo della *test statistic* non cambia!

---

**\[Esempio\]** Un sample di 22 soggetti ha pranzato mentre giocava al pc. Sono stati calcolati i grammi di biscotti consumati come snack in seguito al pranzo. La sample mean è 52.1g, con deviazione standard 45.1g. Chi non ha giocato al pc, invece, ha consumato in media 27.1g di biscotti, con una deviazione standard di 26.4g. Anche in questo caso il sample size è 22. Esiste una differenza significativa fra i due gruppi nel numero di biscotti assunti in media?

In [3]:
n_1 = 22
n_2 = 22
xbar_1 = 52.1
xbar_2 = 27.1
std_1 = 45.1
std_2 = 26.4

sample_1 = sample_creator(n_1, xbar_1, std_1)
sample_2 = sample_creator(n_2, xbar_2, std_2)

print(sample_1.mean(), sample_1.std(ddof = 1))
print(sample_2.mean(), sample_2.std(ddof = 1))

52.1 45.099999999999994
27.10000000000001 26.399999999999995


In [4]:
h0 = 0

test_statistic, p_value, df = sm.stats.ttest_ind(
    x1 = sample_1,
    x2 = sample_2,
    value = h0,
    alternative = "two-sided",
    usevar = "unequal")

test_statistic, p_value, df

(2.2438451596344318, 0.03148839749645822, 33.879263805934805)

---

**[Esempio]** Differenza in media fra pressione sanguigna fra uomo e donna.

In [5]:
bp = pd.read_csv("../data/blood-pressure.csv")

In [6]:
bp_males = bp["bp_after"][bp["sex"] == "Male"]
bp_females = bp["bp_after"][bp["sex"] == "Female"]

_Check normality_

In [7]:
stats.shapiro(bp_males.values - bp_females.values)

(0.98586106300354, 0.7147841453552246)

_Il test non è significativo, quindi la sampling distribution segue andamento normale._

_Check Homogeneity of Variance_

In [8]:
stats.levene(bp_males, bp_females, center = "mean")

LeveneResult(statistic=5.865854141268659, pvalue=0.01695904277978066)

_I gruppi presentano variabilità diverse, forse il t-test non sarà affidabile._

In [9]:
stats.ttest_ind(
    bp["bp_after"][bp["sex"] == "Male"],
    bp["bp_after"][bp["sex"] == "Female"])

Ttest_indResult(statistic=3.3479506182111387, pvalue=0.0010930222986154283)

_Possiamo rigettare l'ipotesi nulla, c'è differenza fra le medie dei gruppi._

---

**[Esempio]** Ad un test per il PTSD, molti mentono. Abbiamo 49 veterani che cercano solo compensation e 70 che cercano un trattamento. Vogliamo capire se c'è differenza fra gli score medi dei due gruppi. Nel gruppo compensation abbiamo una sample mean di 9.76 e dev. std. di 4.90, mentre nel gruppo treatment abbiamo una sample mean di 6.48 e una dev. std. di 3.49.

In [10]:
n_1 = 49
xbar_1 = 9.76
s_1 = 4.90

n_2 = 70
xbar_2 = 6.48
s_2 = 3.49

sample_1 = sample_creator(n_1, xbar_1, s_1)
sample_2 = sample_creator(n_2, xbar_2, s_2)

print(sample_1.mean(), sample_1.std(ddof=1))
print(sample_2.mean(), sample_2.std(ddof=1))

9.76 4.9
6.480000000000001 3.4899999999999998


In [11]:
def t_test_two_means_pooled(sample_1, sample_2):
    n_1, n_2 = len(sample_1), len(sample_2)
    pooled_variance = ((n_1-1)*np.var(sample_1,ddof=1)+\
                       (n_2-1)*np.var(sample_2, ddof = 1))\
                        /(n_1+n_2-2)
    std_error = np.sqrt(pooled_variance)*np.sqrt((1/n_1)+(1/n_2))
    point_estimate = sample_1.mean() - sample_2.mean()
    test_statistic = point_estimate/std_error
    return 2 * stats.t.sf(x = test_statistic, df = n_1+n_2-2)

In [12]:
t_test_two_means_pooled(sample_1, sample_2)

4.047130771605055e-05

Ci sono prove molto forti sulla differenza fra le due medie!

---

**[Esempio]** Si vuole studiare l'efficacia di un antidoto per veleno di serpente. Lo si testa sui maiali, creando un tratment group da 9 soggetti e un placebo group da 8 soggetti. La variabile da monitorare sarà la modifica in volume, per capire se l'antidoto sia efficace contro il gonfiore. 

I dati sono i seguenti, dove il gruppo 1 è il treatment, mentre il gruppo 2 è il placebo.

In [13]:
xbar_1 = 203.33
s_1 = 56.18
n_1 = 9

xbar_2 = 201.25
s_2 = 112.62
n_2 = 8

sample_1 = sample_creator(n_1, xbar_1, s_1)
sample_2 = sample_creator(n_2, xbar_2, s_2)

In [14]:
def t_test_two_means_welch(sample_1, sample_2):
    var_1, var_2 = np.var(sample_1,ddof=1), np.var(sample_2,ddof=1)
    n_1, n_2 = len(sample_1), len(sample_2)
    std_error = np.sqrt((var_1/n_1) + (var_2/n_2))
    point_estimate = np.mean(sample_1) - np.mean(sample_2)
    test_statistic = point_estimate/std_error
    num_df = ((var_1/n_1)+(var_2/n_2))**2
    den_df = ((1/(n_1-1))*((var_1/n_1)**2))\
                + (((1/(n_2-1)))*((var_2/n_2)**2))
    df = num_df/den_df
    
    return 2 * stats.t.sf(x = test_statistic, df = df)

In [15]:
t_test_two_means_welch(sample_1, sample_2)

0.9632265708261043

Non possiamo rigettare l'ipotesi nulla secondo cui le due medie sono coincidenti. In altre parole, non c'è evidenza di **treatment effect**.

---